# 09d - Temporal Analysis Models
 
**Objetivo**: Desarrollar modelos de análisis temporal para entender la evolución del riesgo de Alzheimer a lo largo del tiempo
 
**Componentes principales**:
- Análisis de tendencias temporales en biomarcadores
- Modelos de series de tiempo para predicción de progresión
- Análisis de patrones temporales en actividad/sueño
- Detección de cambios significativos en el tiempo

---

## Importar librerías

In [ ]:
import sys
import os
sys.path.append('../scripts/modeling')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Importar módulos personalizados
from temporal_modeling import TemporalAnalyzer, TimeSeriesPredictor
from model_utils import ModelTracker

import mlflow
import mlflow.sklearn


In [ ]:
# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Configuración MLflow
mlflow.set_experiment("alzheimer_temporal_analysis")

# Inicializar tracker
tracker = ModelTracker(experiment_name="alzheimer_temporal_analysis")

print("🔧 MLflow configurado para Análisis Temporal")


## Carga y Preparación de Datos Temporales

In [ ]:
# Cargar datos procesados
try:
    df = pd.read_csv('../data/processed/integrated_features_final.csv')
    print(f"📊 Dataset cargado: {df.shape}")
    
    # Verificar si hay información temporal
    temporal_columns = [col for col in df.columns if any(x in col.lower() for x in 
                       ['date', 'time', 'visit', 'month', 'year', 'baseline', 'followup'])]
    
    print(f"🕐 Columnas temporales detectadas: {len(temporal_columns)}")
    if temporal_columns:
        print("📋 Columnas temporales disponibles:")
        for col in temporal_columns[:5]:
            print(f"   • {col}")
    
except FileNotFoundError:
    print("❌ Error: Archivo de features no encontrado")
    print("💡 Asegúrate de ejecutar el notebook 03_feature_engineering_master.ipynb primero")


In [ ]:
# Inicializar analizador temporal
temporal_analyzer = TemporalAnalyzer()

# Simular datos temporales si no existen (común en datasets cross-sectional)
if not temporal_columns:
    print("⚠️  No se detectaron columnas temporales explícitas")
    print("🔄 Generando estructura temporal simulada basada en patrones de datos")
    
    # Crear estructura temporal simulada
    np.random.seed(42)
    n_patients = df['composite_risk_score'].notna().sum()
    
    # Simular múltiples visitas por paciente
    visits_per_patient = np.random.poisson(3, n_patients) + 1  # 1-6 visitas
    
    temporal_data = []
    patient_id = 0
    
    for i, n_visits in enumerate(visits_per_patient[:1000]):  # Limitar para demo
        base_risk = df.loc[df['composite_risk_score'].notna()].iloc[i]['composite_risk_score']
        
        for visit in range(n_visits):
            # Simular progresión temporal
            months_elapsed = visit * 6  # Visitas cada 6 meses
            risk_progression = base_risk + (visit * 0.05) + np.random.normal(0, 0.02)
            risk_progression = np.clip(risk_progression, 0, 1)
            
            temporal_data.append({
                'patient_id': patient_id,
                'visit_number': visit,
                'months_elapsed': months_elapsed,
                'composite_risk_score': risk_progression,
                'risk_velocity': 0.05 + np.random.normal(0, 0.01) if visit > 0 else 0
            })
        
        patient_id += 1
    
    df_temporal = pd.DataFrame(temporal_data)
    print(f"📊 Datos temporales simulados: {df_temporal.shape}")
    print(f"👥 Pacientes únicos: {df_temporal['patient_id'].nunique()}")
    print(f"🔄 Visitas promedio por paciente: {df_temporal.groupby('patient_id').size().mean():.1f}")

else:
    # Usar datos temporales reales
    df_temporal = df.copy()
    print("✅ Usando datos temporales reales del dataset")

## Análisis exploratorio temporal

In [ ]:
# Análisis exploratorio temporal
with mlflow.start_run(run_name="temporal_exploration"):
    mlflow.set_tag("analysis_type", "exploratory")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Evolución del riesgo promedio por visita
    if 'visit_number' in df_temporal.columns:
        risk_by_visit = df_temporal.groupby('visit_number')['composite_risk_score'].agg(['mean', 'std']).reset_index()
        
        axes[0,0].errorbar(risk_by_visit['visit_number'], risk_by_visit['mean'], 
                          yerr=risk_by_visit['std'], capsize=5, marker='o')
        axes[0,0].set_title('Evolución del Riesgo Promedio por Visita')
        axes[0,0].set_xlabel('Número de Visita')
        axes[0,0].set_ylabel('Score de Riesgo Promedio')
        axes[0,0].grid(True, alpha=0.3)
        
        # Registrar métricas
        mlflow.log_metric("max_visits", df_temporal['visit_number'].max())
        mlflow.log_metric("avg_risk_increase_per_visit", 
                         (risk_by_visit['mean'].iloc[-1] - risk_by_visit['mean'].iloc[0]) / len(risk_by_visit))
    
    # 2. Distribución de velocidad de cambio
    if 'risk_velocity' in df_temporal.columns:
        velocities = df_temporal[df_temporal['risk_velocity'] != 0]['risk_velocity']
        axes[0,1].hist(velocities, bins=30, alpha=0.7, color='lightcoral')
        axes[0,1].set_title('Distribución de Velocidad de Cambio del Riesgo')
        axes[0,1].set_xlabel('Velocidad de Cambio')
        axes[0,1].set_ylabel('Frecuencia')
        
        mlflow.log_metric("mean_risk_velocity", velocities.mean())
        mlflow.log_metric("std_risk_velocity", velocities.std())
    
    # 3. Trayectorias individuales (muestra)
    sample_patients = df_temporal['patient_id'].unique()[:10]
    for patient in sample_patients:
        patient_data = df_temporal[df_temporal['patient_id'] == patient]
        if len(patient_data) > 1:
            axes[1,0].plot(patient_data['months_elapsed'], patient_data['composite_risk_score'], 
                          alpha=0.6, marker='o', markersize=3)
    
    axes[1,0].set_title('Trayectorias Individuales de Riesgo (Muestra)')
    axes[1,0].set_xlabel('Meses Transcurridos')
    axes[1,0].set_ylabel('Score de Riesgo')
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Correlación temporal
    if len(df_temporal) > 100:
        # Calcular autocorrelación por paciente
        autocorr_data = []
        for patient in df_temporal['patient_id'].unique()[:50]:
            patient_data = df_temporal[df_temporal['patient_id'] == patient].sort_values('months_elapsed')
            if len(patient_data) >= 3:
                risk_series = patient_data['composite_risk_score'].values
                autocorr = np.corrcoef(risk_series[:-1], risk_series[1:])[0,1]
                if not np.isnan(autocorr):
                    autocorr_data.append(autocorr)
        
        if autocorr_data:
            axes[1,1].hist(autocorr_data, bins=20, alpha=0.7, color='lightgreen')
            axes[1,1].set_title('Distribución de Autocorrelación')
            axes[1,1].set_xlabel('Correlación lag-1')
            axes[1,1].set_ylabel('Frecuencia')
            
            mlflow.log_metric("mean_autocorrelation", np.mean(autocorr_data))
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Análisis exploratorio temporal completado")


## Análisis de tendencias con TemporalAnalyzer

In [ ]:
# Análisis de tendencias con TemporalAnalyzer
with mlflow.start_run(run_name="trend_analysis"):
    mlflow.set_tag("analysis_type", "trend_detection")
    
    # Detectar tendencias por paciente
    trend_results = temporal_analyzer.detect_trends(
        df_temporal, 
        patient_col='patient_id',
        time_col='months_elapsed',
        value_col='composite_risk_score'
    )
    
    print("📈 ANÁLISIS DE TENDENCIAS")
    print("=" * 40)
    print(f"Pacientes analizados: {len(trend_results)}")
    
    # Clasificar tendencias
    increasing_trends = sum(1 for r in trend_results if r['slope'] > 0.01)
    stable_trends = sum(1 for r in trend_results if abs(r['slope']) <= 0.01)
    decreasing_trends = sum(1 for r in trend_results if r['slope'] < -0.01)
    
    print(f"Tendencias crecientes: {increasing_trends} ({increasing_trends/len(trend_results)*100:.1f}%)")
    print(f"Tendencias estables: {stable_trends} ({stable_trends/len(trend_results)*100:.1f}%)")
    print(f"Tendencias decrecientes: {decreasing_trends} ({decreasing_trends/len(trend_results)*100:.1f}%)")
    
    # Registrar métricas
    mlflow.log_metric("patients_with_increasing_risk", increasing_trends)
    mlflow.log_metric("patients_with_stable_risk", stable_trends)
    mlflow.log_metric("patients_with_decreasing_risk", decreasing_trends)
    mlflow.log_metric("median_slope", np.median([r['slope'] for r in trend_results]))
    
    # Visualizar distribución de pendientes
    slopes = [r['slope'] for r in trend_results]
    plt.figure(figsize=(10, 6))
    plt.hist(slopes, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    plt.axvline(np.median(slopes), color='red', linestyle='--', label=f'Mediana: {np.median(slopes):.4f}')
    plt.title('Distribución de Pendientes de Tendencia')
    plt.xlabel('Pendiente (cambio en riesgo por mes)')
    plt.ylabel('Frecuencia')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## Modelos predictivos temporales

In [ ]:
# Modelos predictivos temporales
predictor = TimeSeriesPredictor()

with mlflow.start_run(run_name="temporal_prediction"):
    mlflow.set_tag("model_type", "time_series_prediction")
    
    # Seleccionar pacientes con suficientes datos temporales
    patients_with_data = df_temporal.groupby('patient_id').size()
    eligible_patients = patients_with_data[patients_with_data >= 3].index
    
    print(f"👥 Pacientes elegibles para predicción: {len(eligible_patients)}")
    
    # Entrenar modelos predictivos
    prediction_results = []
    
    for patient in eligible_patients[:100]:  # Limitar para demo
        patient_data = df_temporal[df_temporal['patient_id'] == patient].sort_values('months_elapsed')
        
        if len(patient_data) >= 4:  # Mínimo 4 puntos
            # Dividir en train/test temporal
            split_point = len(patient_data) - 1
            train_data = patient_data.iloc[:split_point]
            test_data = patient_data.iloc[split_point:]
            
            # Predicción simple (tendencia linear)
            prediction = predictor.predict_next_value(
                train_data['months_elapsed'].values,
                train_data['composite_risk_score'].values,
                test_data['months_elapsed'].iloc[0]
            )
            
            actual = test_data['composite_risk_score'].iloc[0]
            error = abs(prediction - actual)
            
            prediction_results.append({
                'patient_id': patient,
                'predicted': prediction,
                'actual': actual,
                'error': error,
                'relative_error': error / actual if actual > 0 else 0
            })
    
    # Análisis de resultados de predicción
    if prediction_results:
        pred_df = pd.DataFrame(prediction_results)
        
        mae = pred_df['error'].mean()
        rmse = np.sqrt((pred_df['error'] ** 2).mean())
        mape = pred_df['relative_error'].mean() * 100
        
        print(f"📊 RESULTADOS DE PREDICCIÓN TEMPORAL")
        print(f"MAE: {mae:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAPE: {mape:.2f}%")
        
        # Registrar métricas
        mlflow.log_metric("temporal_mae", mae)
        mlflow.log_metric("temporal_rmse", rmse)
        mlflow.log_metric("temporal_mape", mape)
        mlflow.log_metric("predictions_made", len(prediction_results))
        
        # Visualizar predicciones vs actual
        plt.figure(figsize=(10, 6))
        plt.scatter(pred_df['actual'], pred_df['predicted'], alpha=0.6)
        plt.plot([pred_df['actual'].min(), pred_df['actual'].max()], 
                [pred_df['actual'].min(), pred_df['actual'].max()], 
                'r--', label='Predicción perfecta')
        plt.xlabel('Valor Real')
        plt.ylabel('Valor Predicho')
        plt.title('Predicciones Temporales vs Valores Reales')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

## Análisis de patrones de actividad temporal (si disponible) y Detección de cambios significativos (change points)

In [ ]:
# Análisis de patrones de actividad temporal (si disponible)
activity_columns = [col for col in df.columns if 'sleep' in col.lower() or 'activity' in col.lower()]

if activity_columns:
    with mlflow.start_run(run_name="activity_temporal_patterns"):
        mlflow.set_tag("analysis_type", "activity_patterns")
        
        print("🏃 ANÁLISIS DE PATRONES TEMPORALES DE ACTIVIDAD")
        print("=" * 55)
        
        # Seleccionar features clave de actividad
        key_activity_features = activity_columns[:5]  # Top 5 features
        
        # Simular variabilidad temporal en actividad
        activity_temporal = []
        
        for patient in eligible_patients[:50]:  # Muestra reducida
            patient_base = df[df.index % len(eligible_patients) == patient % len(eligible_patients)]
            
            for visit in range(3):  # 3 visitas simuladas
                activity_row = {
                    'patient_id': patient,
                    'visit': visit,
                    'months_elapsed': visit * 6
                }
                
                # Simular cambios temporales en actividad
                for feature in key_activity_features:
                    if feature in patient_base.columns:
                        base_value = patient_base[feature].iloc[0] if not patient_base[feature].isna().iloc[0] else 0
                        # Añadir variabilidad temporal y tendencia
                        temporal_change = base_value + (visit * 0.1 * np.random.normal(0, 0.5))
                        activity_row[feature] = temporal_change
                
                activity_temporal.append(activity_row)
        
        activity_df = pd.DataFrame(activity_temporal)
        
        # Análizar cambios temporales en actividad
        for feature in key_activity_features:
            if feature in activity_df.columns:
                activity_trends = activity_df.groupby('visit')[feature].mean()
                trend_slope = np.polyfit(activity_trends.index, activity_trends.values, 1)[0]
                
                mlflow.log_metric(f"activity_{feature}_trend_slope", trend_slope)
                print(f"📊 {feature}: tendencia = {trend_slope:.4f}")
        
        # Visualizar patrones temporales de actividad
        if len(key_activity_features) >= 2:
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            
            # Evolución temporal del primer feature
            feature1 = key_activity_features[0]
            activity_evolution = activity_df.groupby('visit')[feature1].agg(['mean', 'std']).reset_index()
            
            axes[0].errorbar(activity_evolution['visit'], activity_evolution['mean'], 
                           yerr=activity_evolution['std'], capsize=5, marker='o')
            axes[0].set_title(f'Evolución Temporal: {feature1}')
            axes[0].set_xlabel('Visita')
            axes[0].set_ylabel('Valor Promedio')
            axes[0].grid(True, alpha=0.3)
            
            # Correlación entre features de actividad a lo largo del tiempo
            if len(key_activity_features) >= 2:
                feature2 = key_activity_features[1]
                for visit in activity_df['visit'].unique():
                    visit_data = activity_df[activity_df['visit'] == visit]
                    if len(visit_data) > 5:
                        axes[1].scatter(visit_data[feature1], visit_data[feature2], 
                                      alpha=0.6, label=f'Visita {visit}')
                
                axes[1].set_xlabel(feature1)
                axes[1].set_ylabel(feature2)
                axes[1].set_title('Correlación Temporal entre Features de Actividad')
                axes[1].legend()
                axes[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()

In [ ]:
# Detección de cambios significativos (change points)
with mlflow.start_run(run_name="change_point_detection"):
    mlflow.set_tag("analysis_type", "change_point_detection")
    
    # Detectar cambios significativos en las trayectorias
    changepoints_detected = []
    
    for patient in eligible_patients[:50]:
        patient_data = df_temporal[df_temporal['patient_id'] == patient].sort_values('months_elapsed')
        
        if len(patient_data) >= 5:
            risk_values = patient_data['composite_risk_score'].values
            
            # Detección simple de cambios (diferencias significativas)
            changes = temporal_analyzer.detect_changepoints(risk_values)
            
            if changes:
                changepoints_detected.append({
                    'patient_id': patient,
                    'n_changepoints': len(changes),
                    'changepoint_positions': changes
                })
    
    print(f"🔍 DETECCIÓN DE PUNTOS DE CAMBIO")
    print("=" * 40)
    print(f"Pacientes con cambios detectados: {len(changepoints_detected)}")
    
    if changepoints_detected:
        n_changes = [cp['n_changepoints'] for cp in changepoints_detected]
        print(f"Promedio de cambios por paciente: {np.mean(n_changes):.2f}")
        print(f"Máximo de cambios detectados: {max(n_changes)}")
        
        mlflow.log_metric("patients_with_changepoints", len(changepoints_detected))
        mlflow.log_metric("avg_changepoints_per_patient", np.mean(n_changes))
        mlflow.log_metric("max_changepoints", max(n_changes))
        
        # Visualizar distribución de cambios
        plt.figure(figsize=(10, 6))
        plt.hist(n_changes, bins=range(max(n_changes)+2), alpha=0.7, color='orange', edgecolor='black')
        plt.title('Distribución de Puntos de Cambio por Paciente')
        plt.xlabel('Número de Puntos de Cambio')
        plt.ylabel('Frecuencia')
        plt.grid(True, alpha=0.3)
        plt.show()

## Resumen del análisis temporal

In [ ]:
# Resumen del análisis temporal
print("\n" + "="*60)
print("📊 RESUMEN DEL ANÁLISIS TEMPORAL")
print("="*60)

temporal_summary = {
    'Total_Observaciones_Temporales': len(df_temporal),
    'Pacientes_Únicos': df_temporal['patient_id'].nunique(),
    'Visitas_Promedio_por_Paciente': df_temporal.groupby('patient_id').size().mean(),
    'Rango_Temporal_Meses': df_temporal['months_elapsed'].max() - df_temporal['months_elapsed'].min(),
}

if 'trend_results' in locals():
    temporal_summary['Pacientes_con_Tendencia_Creciente'] = increasing_trends
    temporal_summary['Pacientes_con_Tendencia_Estable'] = stable_trends

if 'prediction_results' in locals():
    temporal_summary['Predicciones_Realizadas'] = len(prediction_results)
    temporal_summary['Error_Promedio_Predicción'] = f"{mae:.4f}"

if changepoints_detected:
    temporal_summary['Pacientes_con_Cambios_Detectados'] = len(changepoints_detected)

for key, value in temporal_summary.items():
    print(f"🎯 {key}: {value}")

# Guardar resultados temporales
if 'df_temporal' in locals():
    df_temporal.to_csv('../data/processed/temporal_analysis_results.csv', index=False)
    print("\n📁 Resultados temporales guardados en: ../data/processed/temporal_analysis_results.csv")

print("\n✅ Análisis temporal completado exitosamente")
print("📊 Modelos temporales desarrollados y evaluados")
print("🔄 Listo para integración con otros modelos en la fase de ensemble")

---

**Abraham Tartalos**